In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv("E:/DS/Datasets/urdu-sentiment-corpus-v1.tsv",delimiter='\t')

In [3]:
df.head()

,Tweet,Class
0,میں نے ایٹم بم بنایا ھے ۔۔۔۔او بھائی ایٹم بمب ...,P
1,چندے سے انقلاب اور عمران خان وزیر اعظم نہیں بن...,N
2,ٹویٹر کا خیال کیسے آیا ؟,O
3,"سرچ انجن گوگل کے نائب صدر نے فضا میں ، 130,000...",P
4,ابھی تک اسکی لہریں کبھی کبھی آ جاتی ہیں یار :أْ,P


In [4]:
df.Class.value_counts()

N    499
P    480
O     20
Name: Class, dtype: int64

In [5]:
df = df[df.Class != "O"]

In [6]:
df.Class.value_counts()

N    499
P    480
Name: Class, dtype: int64

In [7]:
df.head()

,Tweet,Class
0,میں نے ایٹم بم بنایا ھے ۔۔۔۔او بھائی ایٹم بمب ...,P
1,چندے سے انقلاب اور عمران خان وزیر اعظم نہیں بن...,N
3,"سرچ انجن گوگل کے نائب صدر نے فضا میں ، 130,000...",P
4,ابھی تک اسکی لہریں کبھی کبھی آ جاتی ہیں یار :أْ,P
5,گندی زبان اور گٹر جیسے دماغ والے جاهل جیالے ه...,N


In [8]:
stop_words = []
with open("E:/DS/Datasets/urdu_stopwords.txt",'r',encoding="utf8") as file:
    for item in file:
        stop_words.append(item.partition('\n')[0])

In [10]:
import re
corpus=[]
for i in range(len(df)):
    doc = re.sub('[^ا-ے]',' ',df.iloc[:,0].values[i])
    doc = doc.split()
    doc = [word for word in doc if not word in stop_words]
    doc =' '.join(doc)
    if doc is not None:
        corpus.append(doc)
num_words = len(corpus) 
num_words

980

In [11]:
df['corpous'] = corpus
df.head()

,Tweet,Class,corpous
0,میں نے ایٹم بم بنایا ھے ۔۔۔۔او بھائی ایٹم بمب ...,P,ایٹم بم بنایا ھے بھا ی ایٹم بمب کوٹ لکھپت اتفا...
1,چندے سے انقلاب اور عمران خان وزیر اعظم نہیں بن...,N,چندے انقلاب عمران خان وزیر اعظم بن سکتے
3,"سرچ انجن گوگل کے نائب صدر نے فضا میں ، 130,000...",P,سرچ انجن گوگل ب صدر فضا فٹ بلندی چھلانگ عالمی ...
4,ابھی تک اسکی لہریں کبھی کبھی آ جاتی ہیں یار :أْ,P,اسکی لہریں یار ْ
5,گندی زبان اور گٹر جیسے دماغ والے جاهل جیالے ه...,N,گندی زبان گٹر دماغ جاهل جیالے هو جیالا هو جاهل...


In [14]:
df.to_csv("E:/DS/Datasets/urdu-sentiment.csv",index=False)

In [12]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

In [13]:
trainSize = int(df.shape[0]*0.75)
trainSize

735

In [278]:
trainSize = int(df.shape[0]*0.75)

X_train = df.corpous.iloc[:trainSize].values
y_train = df.Class.iloc[:trainSize].values

X_test = df.corpous.iloc[trainSize:].values
y_test = df.Class.iloc[trainSize:].values

tokenizer = Tokenizer(num_words)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train,maxlen=128,truncating='post',padding='post')
X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test,maxlen=128,truncating='pre',padding='pre')

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [281]:
from keras.models import Sequential
from sklearn.metrics import classification_report
from keras.layers import Embedding,LSTM,SimpleRNN,GRU,Dense,Input,Bidirectional

In [282]:
X = X_train
X = np.reshape(X, (735, 1, 128))
y = y_train

In [283]:
input_shape = (1, 128)
model = Sequential([
    Input(shape=input_shape),
    SimpleRNN(200, dropout=0.3, activation='tanh', return_sequences=True),
    SimpleRNN(100, dropout=0.3, activation='tanh'),
    Dense(1,'sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X,y,epochs=100,batch_size=60,verbose=2)

y_pred = model.predict(np.reshape(X_test, (245, 1, 128)))
y_hat = [1 if y >=0.5 else 0 for y in y_pred]

print(classification_report(y_test,y_hat))

Model: "sequential_45"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_80 (SimpleRNN)   (None, 1, 200)            65800     
                                                                 
 simple_rnn_81 (SimpleRNN)   (None, 100)               30100     
                                                                 
 dense_50 (Dense)            (None, 1)                 101       
                                                                 
Total params: 96,001
Trainable params: 96,001
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
13/13 - 2s - loss: 0.7368 - accuracy: 0.5374 - 2s/epoch - 180ms/step
Epoch 2/100
13/13 - 0s - loss: 0.7373 - accuracy: 0.5224 - 67ms/epoch - 5ms/step
Epoch 3/100
13/13 - 0s - loss: 0.7366 - accuracy: 0.5020 - 69ms/epoch - 5ms/step
Epoch 4/100
13/13 - 0s - loss: 0.7337 - accuracy: 0.5238 - 62ms/e

Epoch 91/100
13/13 - 0s - loss: 0.6756 - accuracy: 0.5782 - 73ms/epoch - 6ms/step
Epoch 92/100
13/13 - 0s - loss: 0.6652 - accuracy: 0.5796 - 67ms/epoch - 5ms/step
Epoch 93/100
13/13 - 0s - loss: 0.6793 - accuracy: 0.5578 - 63ms/epoch - 5ms/step
Epoch 94/100
13/13 - 0s - loss: 0.6700 - accuracy: 0.5741 - 66ms/epoch - 5ms/step
Epoch 95/100
13/13 - 0s - loss: 0.6717 - accuracy: 0.5578 - 64ms/epoch - 5ms/step
Epoch 96/100
13/13 - 0s - loss: 0.6741 - accuracy: 0.5701 - 64ms/epoch - 5ms/step
Epoch 97/100
13/13 - 0s - loss: 0.6877 - accuracy: 0.5483 - 64ms/epoch - 5ms/step
Epoch 98/100
13/13 - 0s - loss: 0.6728 - accuracy: 0.5878 - 65ms/epoch - 5ms/step
Epoch 99/100
13/13 - 0s - loss: 0.6766 - accuracy: 0.5728 - 63ms/epoch - 5ms/step
Epoch 100/100
13/13 - 0s - loss: 0.6864 - accuracy: 0.5524 - 64ms/epoch - 5ms/step
8/8 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.52      0.69      0.59       120
           1       

In [284]:
input_shape = (1, 128)
model = Sequential([
    Input(shape=input_shape),
    GRU(200, dropout=0.3, activation='tanh', return_sequences=True),
    GRU(100, dropout=0.3, activation='tanh'),
    Dense(1,'sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X,y,epochs=100,batch_size=60,verbose=2)

y_pred = model.predict(np.reshape(X_test, (245, 1, 128)))
y_hat = [1 if y >=0.5 else 0 for y in y_pred]

print(classification_report(y_test,y_hat))

Model: "sequential_46"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_2 (GRU)                 (None, 1, 200)            198000    
                                                                 
 gru_3 (GRU)                 (None, 100)               90600     
                                                                 
 dense_51 (Dense)            (None, 1)                 101       
                                                                 
Total params: 288,701
Trainable params: 288,701
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
13/13 - 7s - loss: 0.7089 - accuracy: 0.4952 - 7s/epoch - 556ms/step
Epoch 2/100
13/13 - 0s - loss: 0.6884 - accuracy: 0.5524 - 108ms/epoch - 8ms/step
Epoch 3/100
13/13 - 0s - loss: 0.7189 - accuracy: 0.5088 - 108ms/epoch - 8ms/step
Epoch 4/100
13/13 - 0s - loss: 0.6942 - accuracy: 0.5333 - 10

Epoch 90/100
13/13 - 0s - loss: 0.6581 - accuracy: 0.6068 - 119ms/epoch - 9ms/step
Epoch 91/100
13/13 - 0s - loss: 0.6678 - accuracy: 0.5891 - 112ms/epoch - 9ms/step
Epoch 92/100
13/13 - 0s - loss: 0.6533 - accuracy: 0.5959 - 108ms/epoch - 8ms/step
Epoch 93/100
13/13 - 0s - loss: 0.6661 - accuracy: 0.5918 - 111ms/epoch - 9ms/step
Epoch 94/100
13/13 - 0s - loss: 0.6654 - accuracy: 0.5755 - 108ms/epoch - 8ms/step
Epoch 95/100
13/13 - 0s - loss: 0.6664 - accuracy: 0.5782 - 105ms/epoch - 8ms/step
Epoch 96/100
13/13 - 0s - loss: 0.6674 - accuracy: 0.5878 - 128ms/epoch - 10ms/step
Epoch 97/100
13/13 - 0s - loss: 0.6677 - accuracy: 0.5850 - 138ms/epoch - 11ms/step
Epoch 98/100
13/13 - 0s - loss: 0.6528 - accuracy: 0.6204 - 113ms/epoch - 9ms/step
Epoch 99/100
13/13 - 0s - loss: 0.6629 - accuracy: 0.5918 - 103ms/epoch - 8ms/step
Epoch 100/100
13/13 - 0s - loss: 0.6674 - accuracy: 0.5810 - 106ms/epoch - 8ms/step
8/8 [==============================] - 1s 4ms/step
              precision    recall

In [285]:
model = Sequential([
    Input(shape=input_shape),
    LSTM(200, dropout=0.3, activation='tanh', return_sequences=True),
    LSTM(100, dropout=0.3, activation='tanh'),
    Dense(1,'sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X,y,epochs=100,batch_size=60,verbose=2)

y_pred = model.predict(np.reshape(X_test, (245, 1, 128)))
y_hat = [1 if y >=0.5 else 0 for y in y_pred]

print(classification_report(y_test,y_hat))

Model: "sequential_47"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 1, 200)            263200    
                                                                 
 lstm_5 (LSTM)               (None, 100)               120400    
                                                                 
 dense_52 (Dense)            (None, 1)                 101       
                                                                 
Total params: 383,701
Trainable params: 383,701
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
13/13 - 6s - loss: 0.6982 - accuracy: 0.4816 - 6s/epoch - 456ms/step
Epoch 2/100
13/13 - 0s - loss: 0.6882 - accuracy: 0.5497 - 115ms/epoch - 9ms/step
Epoch 3/100
13/13 - 0s - loss: 0.6917 - accuracy: 0.5224 - 105ms/epoch - 8ms/step
Epoch 4/100
13/13 - 0s - loss: 0.6902 - accuracy: 0.5361 - 11

Epoch 90/100
13/13 - 0s - loss: 0.6597 - accuracy: 0.5918 - 120ms/epoch - 9ms/step
Epoch 91/100
13/13 - 0s - loss: 0.6584 - accuracy: 0.5714 - 110ms/epoch - 8ms/step
Epoch 92/100
13/13 - 0s - loss: 0.6677 - accuracy: 0.5850 - 106ms/epoch - 8ms/step
Epoch 93/100
13/13 - 0s - loss: 0.6666 - accuracy: 0.5864 - 113ms/epoch - 9ms/step
Epoch 94/100
13/13 - 0s - loss: 0.6662 - accuracy: 0.5878 - 118ms/epoch - 9ms/step
Epoch 95/100
13/13 - 0s - loss: 0.6675 - accuracy: 0.5891 - 113ms/epoch - 9ms/step
Epoch 96/100
13/13 - 0s - loss: 0.6637 - accuracy: 0.5755 - 110ms/epoch - 8ms/step
Epoch 97/100
13/13 - 0s - loss: 0.6628 - accuracy: 0.5878 - 107ms/epoch - 8ms/step
Epoch 98/100
13/13 - 0s - loss: 0.6540 - accuracy: 0.5796 - 111ms/epoch - 9ms/step
Epoch 99/100
13/13 - 0s - loss: 0.6788 - accuracy: 0.5551 - 119ms/epoch - 9ms/step
Epoch 100/100
13/13 - 0s - loss: 0.6576 - accuracy: 0.5850 - 117ms/epoch - 9ms/step
8/8 [==============================] - 1s 4ms/step
              precision    recall  

In [287]:
model = Sequential([
    Input(shape=input_shape),
    Bidirectional(LSTM(200, dropout=0.3, activation='tanh', return_sequences=True)),
    Bidirectional(LSTM(100, dropout=0.3, activation='tanh')),
    Dense(1,'sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X,y,epochs=100,batch_size=60,verbose=2)

y_pred = model.predict(np.reshape(X_test, (245, 1, 128)))
y_hat = [1 if y >=0.5 else 0 for y in y_pred]

print(classification_report(y_test,y_hat))

Model: "sequential_48"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (None, 1, 400)           526400    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 200)              400800    
 nal)                                                            
                                                                 
 dense_53 (Dense)            (None, 1)                 201       
                                                                 
Total params: 927,401
Trainable params: 927,401
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
13/13 - 10s - loss: 0.6958 - accuracy: 0.5184 - 10s/epoch - 763ms/step
Epoch 2/100
13/13 - 0s - loss: 0.6923 - accuracy: 0.5442 - 175ms/epoch - 13ms/step
Epoch 3/

Epoch 88/100
13/13 - 0s - loss: 0.6523 - accuracy: 0.5837 - 196ms/epoch - 15ms/step
Epoch 89/100
13/13 - 0s - loss: 0.6280 - accuracy: 0.6177 - 172ms/epoch - 13ms/step
Epoch 90/100
13/13 - 0s - loss: 0.6364 - accuracy: 0.6190 - 238ms/epoch - 18ms/step
Epoch 91/100
13/13 - 0s - loss: 0.6437 - accuracy: 0.6041 - 192ms/epoch - 15ms/step
Epoch 92/100
13/13 - 0s - loss: 0.6350 - accuracy: 0.6231 - 197ms/epoch - 15ms/step
Epoch 93/100
13/13 - 0s - loss: 0.6354 - accuracy: 0.6109 - 250ms/epoch - 19ms/step
Epoch 94/100
13/13 - 0s - loss: 0.6453 - accuracy: 0.5837 - 194ms/epoch - 15ms/step
Epoch 95/100
13/13 - 0s - loss: 0.6173 - accuracy: 0.6531 - 233ms/epoch - 18ms/step
Epoch 96/100
13/13 - 0s - loss: 0.6270 - accuracy: 0.6327 - 199ms/epoch - 15ms/step
Epoch 97/100
13/13 - 0s - loss: 0.6357 - accuracy: 0.6286 - 194ms/epoch - 15ms/step
Epoch 98/100
13/13 - 0s - loss: 0.6346 - accuracy: 0.6204 - 218ms/epoch - 17ms/step
Epoch 99/100
13/13 - 0s - loss: 0.6415 - accuracy: 0.6231 - 189ms/epoch - 15

# Question 2

In [1]:
import numpy as np
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from sklearn.metrics import classification_report
from keras.layers import Embedding,GRU,Dense,Bidirectional

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("E:/DS/Datasets/urdu-sentiment.csv")
df.head()

,Tweet,Class,corpous
0,میں نے ایٹم بم بنایا ھے ۔۔۔۔او بھائی ایٹم بمب ...,P,ایٹم بم بنایا ھے بھا ی ایٹم بمب کوٹ لکھپت اتفا...
1,چندے سے انقلاب اور عمران خان وزیر اعظم نہیں بن...,N,چندے انقلاب عمران خان وزیر اعظم بن سکتے
2,"سرچ انجن گوگل کے نائب صدر نے فضا میں ، 130,000...",P,سرچ انجن گوگل ب صدر فضا فٹ بلندی چھلانگ عالمی ...
3,ابھی تک اسکی لہریں کبھی کبھی آ جاتی ہیں یار :أْ,P,اسکی لہریں یار ْ
4,گندی زبان اور گٹر جیسے دماغ والے جاهل جیالے ه...,N,گندی زبان گٹر دماغ جاهل جیالے هو جیالا هو جاهل...


In [50]:
from gensim.models import Word2Vec

In [60]:
corpus = df.corpous.apply(lambda x: str(x).split()) # tokenizing

model_w2v = Word2Vec(corpus, vector_size=300,  window=5)

model_w2v.train(corpus, total_examples= len(df.corpous), epochs=150)
     

(546495, 1558650)

In [63]:
model_w2v.wv.most_similar(positive='زبان')

[('لوگوں', 0.7340641021728516),
 ('منہ', 0.7099151611328125),
 ('امریکہ', 0.6859112977981567),
 ('گھر', 0.6813095808029175),
 ('جواب', 0.6527116894721985),
 ('اچھی', 0.6017876267433167),
 ('ٹوٹ', 0.5949305891990662),
 ('فا', 0.5865015983581543),
 ('لفظ', 0.5750141739845276),
 ('حق', 0.5624004006385803)]

In [64]:
model_w2v.wv.most_similar(positive='چندہ')

[('مانگنے', 0.752217173576355),
 ('سکتے', 0.690217912197113),
 ('کنٹینر', 0.6728219389915466),
 ('دھرنے', 0.6409998536109924),
 ('تبدیلی', 0.6405375599861145),
 ('دس', 0.6199865937232971),
 ('چندے', 0.5991798639297485),
 ('کیلیے', 0.5940641760826111),
 ('عمران', 0.5888068079948425),
 ('مریم', 0.5807701945304871)]

In [51]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += model_w2v.wv[word].reshape((1, size))
            count += 1.
        except KeyError: # handling the case where the token is not in vocabulary
                         
            continue
    if count != 0.:
        vec /= count
    return vec

In [68]:
wordvec_arrays = np.zeros((len(corpus), 300))

for i in range(len(corpus)):
    wordvec_arrays[i,:] = word_vector(corpus[i], 300)
    
wordvec_df = pd.DataFrame(wordvec_arrays)
wordvec_df.shape

(980, 300)

In [85]:
from sklearn.model_selection import train_test_split
trainSize = int(df.shape[0]*0.75)
num_words = len(corpus)

X_train, X_test, y_train, y_test = train_test_split(df.corpous, df.Class,
                                                    test_size=0.3, random_state=42)

X_train = X_train.astype(str)
X_test  = X_test.astype(str)

tokenizer = Tokenizer(num_words)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train,maxlen=128,truncating='post',padding='post')

X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test,maxlen=128,truncating='pre',padding='pre')

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

numpy.ndarray

In [99]:
model = Sequential([
    Embedding(input_dim=len(wordvec_arrays),
                            output_dim=300,
                            weights=[wordvec_arrays],
                            input_length=128,
                            trainable=False),
    GRU(200, dropout=0.3, activation='tanh', return_sequences=True),
    GRU(100, dropout=0.3, activation='tanh'),
    Dense(1,'sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['Accuracy',
                                                                     'Precision',
                                                                     'Recall',
                                                                   
                                                
                                                                    ])
print(model.summary())

hist = model.fit(X_train,y_train,epochs=30,batch_size=60,verbose=2,validation_data=(X_test,y_test))

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 128, 300)          294000    
                                                                 
 gru_22 (GRU)                (None, 128, 200)          301200    
                                                                 
 gru_23 (GRU)                (None, 100)               90600     
                                                                 
 dense_11 (Dense)            (None, 1)                 101       
                                                                 
Total params: 685,901
Trainable params: 391,901
Non-trainable params: 294,000
_________________________________________________________________
None
Epoch 1/30
12/12 - 18s - loss: 0.7294 - Accuracy: 0.5029 - precision: 0.4899 - recall: 0.4358 - val_loss: 0.7048 - val_Accuracy: 0.4932 - val_precision: 0.4949 - val_re

In [106]:
accuracy = hist.history['Accuracy'][-1]
precision = hist.history['precision'][-1]
recall = hist.history['recall'][-1]
F1Scrore = 2 * (precision * recall) / (precision + recall)
print(f"Accuracy : {accuracy} \nPrecision : {precision} \nRecall : {recall} \nF1Score : {F1Scrore}")

Accuracy : 0.5204081535339355 
Precision : 0.5288461446762085 
Recall : 0.16417910158634186 
F1Score : 0.2505694716854425


In [107]:
 from gensim.models import FastText

In [109]:
corpus = df.corpous.apply(lambda x: str(x).split()) # tokenizing

model_w2v = FastText(corpus, vector_size=300,  window=5)

model_w2v.train(corpus, total_examples= len(df.corpous), epochs=150)
     

(546495, 1558650)

In [110]:
model_w2v.wv.most_similar(positive='زبان')

[('لوگوں', 0.741570234298706),
 ('گھر', 0.6810804009437561),
 ('منہ', 0.6359058618545532),
 ('شکریہ', 0.5818443298339844),
 ('جان', 0.5818175673484802),
 ('فا', 0.5722193717956543),
 ('پتا', 0.5632427930831909),
 ('ویسے', 0.5613620281219482),
 ('امریکہ', 0.5609874129295349),
 ('لفظ', 0.5557683706283569)]

In [111]:
model_w2v.wv.most_similar(positive='چندہ')

[('چندے', 0.81256502866745),
 ('مانگنے', 0.7959380745887756),
 ('سکتے', 0.7338665723800659),
 ('کنٹینر', 0.7073994874954224),
 ('بغیر', 0.652493417263031),
 ('عمران', 0.5894601941108704),
 ('تبدیلی', 0.5885247588157654),
 ('جگہ', 0.5880036950111389),
 ('دھرنے', 0.5824985504150391),
 ('انقلاب', 0.5786046981811523)]

In [112]:
wordvec_arrays = np.zeros((len(corpus), 300))

for i in range(len(corpus)):
    wordvec_arrays[i,:] = word_vector(corpus[i], 300)
    
wordvec_df = pd.DataFrame(wordvec_arrays)
wordvec_df.shape

(980, 300)

In [113]:
from sklearn.model_selection import train_test_split
trainSize = int(df.shape[0]*0.75)
num_words = len(corpus)

X_train, X_test, y_train, y_test = train_test_split(df.corpous, df.Class,
                                                    test_size=0.3, random_state=42)

X_train = X_train.astype(str)
X_test  = X_test.astype(str)

tokenizer = Tokenizer(num_words)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train,maxlen=128,truncating='post',padding='post')

X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test,maxlen=128,truncating='pre',padding='pre')

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [114]:
model = Sequential([
    Embedding(input_dim=len(wordvec_arrays),
                            output_dim=300,
                            weights=[wordvec_arrays],
                            input_length=128,
                            trainable=False),
    GRU(200, dropout=0.3, activation='tanh', return_sequences=True),
    GRU(100, dropout=0.3, activation='tanh'),
    Dense(1,'sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['Accuracy',
                                                                     'Precision',
                                                                     'Recall',
                                                                   
                                                
                                                                    ])
print(model.summary())

hist = model.fit(X_train,y_train,epochs=30,batch_size=60,verbose=2,validation_data=(X_test,y_test))

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, 128, 300)          294000    
                                                                 
 gru_24 (GRU)                (None, 128, 200)          301200    
                                                                 
 gru_25 (GRU)                (None, 100)               90600     
                                                                 
 dense_12 (Dense)            (None, 1)                 101       
                                                                 
Total params: 685,901
Trainable params: 391,901
Non-trainable params: 294,000
_________________________________________________________________
None
Epoch 1/30
12/12 - 21s - loss: 0.7051 - Accuracy: 0.5000 - precision: 0.4730 - recall: 0.2090 - val_loss: 0.6968 - val_Accuracy: 0.4762 - val_precision: 0.4846 - val_re

In [116]:
accuracy = hist.history['Accuracy'][-2]
precision = hist.history['precision'][-2]
recall = hist.history['recall'][-2]
F1Scrore = 2 * (precision * recall) / (precision + recall)
print(f"Accuracy : {accuracy} \nPrecision : {precision} \nRecall : {recall} \nF1Score : {F1Scrore}")

Accuracy : 0.4693877696990967 
Precision : 0.4530201256275177 
Recall : 0.4029850661754608 
F1Score : 0.4265402758137467


In [1]:
#!pip install allennlp

In [40]:
from elmoformanylangs.elmo import Embedder

In [42]:
e = Embedder('C:/Users/RizzWann/Downloads/Compressed/177')

2023-04-28 14:57:17,687 WARNING: Could not find config.  Trying C:/Users/RizzWann/Downloads/Compressed/177\cnn_50_100_512_4096_sample.json
2023-04-28 14:57:17,689 WARNING: Could not find config.  Trying C:\Users\RizzWann\anaconda3\lib\site-packages\elmoformanylangs\configs\cnn_50_100_512_4096_sample.json
2023-04-28 14:57:17,709 INFO: char embedding size: 3272
2023-04-28 14:57:18,570 INFO: word embedding size: 171020
2023-04-28 14:57:24,619 INFO: Model(
  (token_embedder): ConvTokenEmbedder(
    (word_emb_layer): EmbeddingLayer(
      (embedding): Embedding(171020, 100, padding_idx=3)
    )
    (char_emb_layer): EmbeddingLayer(
      (embedding): Embedding(3272, 50, padding_idx=3269)
    )
    (convolutions): ModuleList(
      (0): Conv1d(50, 32, kernel_size=(1,), stride=(1,))
      (1): Conv1d(50, 32, kernel_size=(2,), stride=(1,))
      (2): Conv1d(50, 64, kernel_size=(3,), stride=(1,))
      (3): Conv1d(50, 128, kernel_size=(4,), stride=(1,))
      (4): Conv1d(50, 256, kernel_size=(5

In [55]:
def word_vector_elmo(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += e.sents2elmo(word)[0][0].reshape((1, size))
            count += 1.
        except KeyError: # handling the case where the token is not in vocabulary
                         
            continue
    if count != 0.:
        vec /= count
    return vec

In [ ]:
corpus = df.corpous.apply(lambda x: str(x).split()) # tokenizing
wordvec_arrays = np.zeros((len(corpus), 1024))


In [ ]:
for i in range(len(corpus)):
    wordvec_arrays[i,:] = word_vector_elmo(corpus[i],1024)
    
wordvec_df = pd.DataFrame(wordvec_arrays)
wordvec_df.shape

In [58]:
e_emb.shape

(1, 1024)

In [61]:
wordvec_df.to_csv("E:/DS/Datasets/elmo_urdu_emb.csv",index=False)

In [14]:
wordvec_arrays = pd.read_csv('E:/DS/Datasets/elmo_urdu_emb.csv')
wordvec_arrays = wordvec_arrays.values
wordvec_arrays.shape

(980, 1024)

In [15]:
from sklearn.model_selection import train_test_split
trainSize = int(df.shape[0]*0.75)
num_words = len(corpus)

X_train, X_test, y_train, y_test = train_test_split(df.corpous, df.Class,
                                                    test_size=0.3, random_state=42)

X_train = X_train.astype(str)
X_test  = X_test.astype(str)

tokenizer = Tokenizer(num_words)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train,maxlen=128,truncating='post',padding='post')

X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test,maxlen=128,truncating='pre',padding='pre')

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [17]:
model = Sequential([
    Embedding(input_dim=len(wordvec_arrays),
                            output_dim=1024,
                            weights=[wordvec_arrays],
                            input_length=128,
                            trainable=False),
    GRU(200, dropout=0.3, activation='tanh', return_sequences=True),
    GRU(100, dropout=0.3, activation='tanh'),
    Dense(1,'sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['Accuracy',
                                                                     'Precision',
                                                                     'Recall',
                                                                   
                                                
                                                                    ])
print(model.summary())

hist = model.fit(X_train,y_train,epochs=10,batch_size=60,verbose=2,validation_data=(X_test,y_test))

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 128, 1024)         1003520   
                                                                 
 gru_8 (GRU)                 (None, 128, 200)          735600    
                                                                 
 gru_9 (GRU)                 (None, 100)               90600     
                                                                 
 dense_4 (Dense)             (None, 1)                 101       
                                                                 
Total params: 1,829,821
Trainable params: 826,301
Non-trainable params: 1,003,520
_________________________________________________________________
None
Epoch 1/10
12/12 - 16s - loss: 0.8337 - Accuracy: 0.5262 - precision: 0.5188 - recall: 0.4119 - val_loss: 0.7386 - val_Accuracy: 0.4966 - val_precision: 0.4966 - val

In [18]:
accuracy = hist.history['Accuracy'][-2]
precision = hist.history['precision'][-2]
recall = hist.history['recall'][-2]
F1Scrore = 2 * (precision * recall) / (precision + recall)
print(f"Accuracy : {accuracy} \nPrecision : {precision} \nRecall : {recall} \nF1Score : {F1Scrore}")

Accuracy : 0.5364431738853455 
Precision : 0.5343511700630188 
Recall : 0.41791045665740967 
F1Score : 0.46901174053507994


In [4]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('C:/Users/RizzWann/Downloads/Compressed/glove.6B/glove.6B.300d.txt',encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [10]:
 print(embeddings_index.get('گھر'))

None


In [5]:
def word_vector_glove(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += embeddings_index.get(word).reshape((1, size))
            count += 1.
        except KeyError: # handling the case where the token is not in vocabulary
                         
            continue
    if count != 0.:
        vec /= count
    return vec

In [ ]:
# create a weight matrix for words in training docs
corpus = df.corpous.apply(lambda x: str(x).split())
vocab_size = len(corpus) + 1
embedding_matrix = np.zeros((vocab_size, 300))
for i in range(len(corpus)):
    embedding_matrix[i,:] = word_vector_glove(corpus[i], 300)
    print(i)
   

In [ ]:
from sklearn.model_selection import train_test_split
trainSize = int(df.shape[0]*0.75)
num_words = len(corpus)

X_train, X_test, y_train, y_test = train_test_split(df.corpous, df.Class,
                                                    test_size=0.3, random_state=42)

X_train = X_train.astype(str)
X_test  = X_test.astype(str)

tokenizer = Tokenizer(num_words)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train,maxlen=128,truncating='post',padding='post')

X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test,maxlen=128,truncating='pre',padding='pre')

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [ ]:
model = Sequential([
    Embedding(input_dim=len(embedding_matrix),
                            output_dim=300,
                            weights=[embedding_matrix],
                            input_length=128,
                            trainable=False),
    GRU(200, dropout=0.3, activation='tanh', return_sequences=True),
    GRU(100, dropout=0.3, activation='tanh'),
    Dense(1,'sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['Accuracy',
                                                                     'Precision',
                                                                     'Recall',
                                                                   
                                                
                                                                    ])
print(model.summary())

hist = model.fit(X_train,y_train,epochs=10,batch_size=60,verbose=2,validation_data=(X_test,y_test))

In [8]:
#!pip install fasttext

     -------------------------------------- 68.8/68.8 kB 156.3 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for fasttext: filename=fasttext-0.9.2-cp39-cp39-win_amd64.whl size=229474 sha256=692331dd3dd86ea82e7f73975dc1e10ac781778a54e750e50b43c41deedcb7bf
  Stored in directory: c:\users\rizzwann\appdata\local\pip\cache\wheels\64\57\bc\1741406019061d5664914b070bd3e71f6244648732bc96109e
Successfully built fasttext


In [9]:
import fasttext

In [12]:
model = fasttext.train_unsupervised('E:DS/check.txt', dim=300, epoch=10)

ValueError: Empty vocabulary. Try a smaller -minCount value.